# Slate Star Codex Reading List

In [1]:
import pandas as pd
import numpy as np

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
data = pd.read_excel('ssc2018public_original.xlsx')
data = data[[i for i in data.columns if 'Unnamed' not in i]]

In [3]:
from string import punctuation

for i in punctuation:
    data['Favoriteblog'] = data['Favoriteblog'].str.replace(i, '')
    data['Favoritepost'] = data['Favoritepost'].str.replace(i, '')
    
data['Favoriteblog'] = data['Favoriteblog'].str.lower()
data['Favoritepost'] = data['Favoritepost'].str.lower()

def fix_title(str1, str2, col):
    data[col] = np.where(data[col] == str1, str2, data[col])

def title_contains(to_replace, replacement, col):
    data[col] = np.where(data[col].str.contains(to_replace), replacement, data[col])

data['Favoriteblog'].fillna('None', inplace=True)

fix_title('blogfefede', 'blog.fefe.de', 'Favoriteblog')
title_contains('instapundit', 'instapundit', 'Favoriteblog')
title_contains('shtetl', 'shtetl optimized', 'Favoriteblog')
title_contains('marginal', 'marginal revolution', 'Favoriteblog')
title_contains('psychiatrist', 'the last psychiatrist', 'Favoriteblog')
title_contains('caring', 'the unit of caring', 'Favoriteblog')
title_contains('wrong', 'less wrong', 'Favoriteblog')
title_contains('wait', 'wait but why', 'Favoriteblog')
title_contains('gelman', 'andrew gelman', 'Favoriteblog')
title_contains('Heartiste', 'chateau heartiste', 'Favoriteblog')
title_contains('zvi', 'zvi', 'Favoriteblog')
title_contains('foxnews', 'fox news', 'Favoriteblog')
title_contains('wdtprs', 'wdtprs', 'Favoriteblog')
title_contains('fr z', 'wdtprs', 'Favoriteblog')
title_contains('mustache', 'mr money mustache', 'Favoriteblog')
title_contains('hours', '80000 hours', 'Favoriteblog')
title_contains('kevin', 'kevin md', 'Favoriteblog')
title_contains('caplan', 'bryan caplan', 'Favoriteblog')

In [4]:
blog_dummies = pd.get_dummies(data['Favoriteblog'], prefix='')
blog_dummies.columns = [i.replace('_', '') for i in blog_dummies.columns]
del blog_dummies['none']
del blog_dummies['None']

blog_dummies = blog_dummies[[i for i in blog_dummies.columns if blog_dummies[i].sum() >= 2]]

In [5]:
from scipy.stats import pearsonr

blogs = []

for i in blog_dummies.columns:
    corr = pearsonr(data['PoliticalSpectrum'].fillna(data['PoliticalSpectrum'].median()), blog_dummies[i])
    blogs.append([i, corr[0], corr[1]])

blogs = pd.DataFrame(blogs, columns=['blog', 'corr', 'pvalue'])

# Blogs Liberals Like

In [6]:
blogs[(blogs['pvalue'] < .01)  & (blogs['corr'] < 0)].sort_values('corr').head(20)[['blog', 'corr']]

,blog,corr
302,thing of things,-0.058271
255,shtetl optimized,-0.056378
297,the unit of caring,-0.054728
189,naked capitalism,-0.048033
30,blog.fefe.de,-0.045023
141,kevin md,-0.038462
317,wait but why,-0.034693


# Blogs Conservatives Like

In [7]:
blogs[blogs.pvalue < .01].sort_values('corr', ascending=False).head(20)[['blog', 'corr']]

,blog,corr
271,steve sailer,0.119845
135,instapundit,0.111025
161,marginal revolution,0.107479
312,unqualified reservations,0.094321
66,econlog,0.088131
314,vox popoli,0.066766
137,isteve,0.054051
46,chateau heartiste,0.048648
327,xenosystems,0.048326
264,social matter,0.048040


# Blogs That Smart People Like (and Don't Like)

In [8]:
data['IQ'] = pd.to_numeric(data['IQ'], errors='coerce')

blogs = []

for i in blog_dummies.columns:
    corr = pearsonr(data['IQ'].fillna(data['IQ'].median()), blog_dummies[i])
    blogs.append([i, corr[0], corr[1]])

blogs = pd.DataFrame(blogs, columns=['blog', 'corr', 'pvalue'])

blogs[blogs.pvalue < .05].sort_values('corr', ascending=False).head(20)[['blog', 'corr']]

,blog,corr
161,marginal revolution,0.034777
91,glibertarianscom,0.023500
189,naked capitalism,-0.028231


In [9]:
data['BigFiveC'] = pd.to_numeric(data['BigFiveC'], errors='coerce')

blogs = []

for i in blog_dummies.columns:
    corr = pearsonr(data['BigFiveC'].fillna(data['BigFiveC'].median()), blog_dummies[i])
    blogs.append([i, corr[0], corr[1]])

blogs = pd.DataFrame(blogs, columns=['blog', 'corr', 'pvalue'])

blogs[blogs.pvalue < .01].sort_values('corr', ascending=False).head(20)[['blog', 'corr']]

,blog,corr
155,longform,0.038827
168,mcmansion hell,0.032436
144,krebs on security,0.030674


In [10]:
data['BigFiveA'] = pd.to_numeric(data['BigFiveA'], errors='coerce')

blogs = []

for i in blog_dummies.columns:
    corr = pearsonr(data['BigFiveA'].fillna(data['BigFiveA'].median()), blog_dummies[i])
    blogs.append([i, corr[0], corr[1]])

blogs = pd.DataFrame(blogs, columns=['blog', 'corr', 'pvalue'])

blogs[blogs.pvalue < .01].sort_values('corr', ascending=False).head(20)[['blog', 'corr']]

,blog,corr
302,thing of things,0.051079
70,edward feser,0.043361
255,shtetl optimized,0.031113
161,marginal revolution,-0.030863
199,overcoming bias,-0.034548


# Top SSC Posts

In [11]:
data['Favoritepost'] = data['Favoritepost'].str.lower()
data['Favoritepost'] = data['Favoritepost'].fillna('none')

title_contains('moloc', 'meditations on moloch', 'Favoritepost')
title_contains('outgroup', 'i can tolerate anything except the outgroup', 'Favoritepost')
title_contains('toxo', 'toxoplasma of rage', 'Favoritepost')
title_contains('rabbit', 'and i show you how deep the rabbit hole goes', 'Favoritepost')
title_contains('brick', 'all in all another brick in the motte', 'Favoritepost')
title_contains('whyaretransgender', 'why are transgender people immune to optical illusions', 'Favoritepost')
title_contains('ssris', 'ssris much more than you wanted to know', 'Favoritepost')
title_contains('cactus', 'universal love, said the cactus person', 'Favoritepost')
title_contains('cost disease', 'considerations on cost disease', 'Favoritepost')
title_contains('thingsthatsome', 'things that sometimes help if you have depression', 'Favoritepost')

In [12]:
post_dummies = pd.get_dummies(data['Favoritepost'], prefix='')
post_dummies.columns = [i.replace('_', '') for i in post_dummies.columns]
del post_dummies['none']

post_dummies = post_dummies[[i for i in post_dummies.columns if post_dummies[i].sum() >= 2]]

In [13]:
print('Votes', '\t', 'Post', '\n')

for title, votes in zip(data['Favoritepost'].value_counts().index[1:],
                        data['Favoritepost'].value_counts()[1:]):

    if votes < 10:
        break
    print(votes, '\t', title.title())

Votes 	 Post 

1201 	 Meditations On Moloch
588 	 I Can Tolerate Anything Except The Outgroup
184 	 Toxoplasma Of Rage
106 	 Considerations On Cost Disease
73 	 Universal Love, Said The Cactus Person
72 	 The Control Group Is Out Of Control
70 	 Untitled
70 	 And I Show You How Deep The Rabbit Hole Goes
67 	 The Categories Were Made For Man Not Man For The Categories
59 	 In Favor Of Niceness Community And Civilization
57 	 Who By Very Slow Decay
53 	 Radicalizing The Romanceless
47 	 You Are Still Crying Wolf
30 	 A Modern Myth
25 	 Book Review Surfing Uncertainty
21 	 Against Murderism
20 	 Different Worlds
20 	 Archipelago And Atomic Communitarianism
20 	 Guided By The Beauty Of Our Weapons
19 	 The Goddess Of Everything Else
19 	 Beware The Man Of One Study
17 	 Nobody Is Perfect Everything Is Commensurable
16 	 My Irb Nightmare
16 	 Unsong
15 	 Social Justice And Words Words Words
15 	 Contra Grant On Exaggerated Differences
14 	 Right Is The New Left
14 	 It Was You Who Made My B

In [14]:
posts = []

for i in post_dummies.columns:
    corr = pearsonr(data['PoliticalSpectrum'].fillna(data['PoliticalSpectrum'].median()), post_dummies[i])
    posts.append([i, corr[0], corr[1]])

posts = pd.DataFrame(posts, columns=['posts', 'corr', 'pvalue'])

# SSC Posts Liberals Like

Because of all the missing data, the correlations we'll see from this point forward are quite small.

In [15]:
posts[(posts['pvalue'] < .01)  & (posts['corr'] < 0)].sort_values('corr').head(20)[['posts', 'corr']]

,posts,corr
106,meditations on moloch,-0.070113
151,the categories were made for man not man for t...,-0.050850
165,the study of anglophysics,-0.030537


# SSC Posts Conservatives Like

In [16]:
posts[(posts['pvalue'] < .01) & (posts['corr'] > 0)].sort_values('corr', ascending=False).head(20)[['posts', 'corr']]

,posts,corr
89,i can tolerate anything except the outgroup,0.084215
191,you are still crying wolf,0.068565
136,right is the new left,0.033939
179,untitled,0.032614
40,chametz,0.032325
17,all in all another brick in the motte,0.030548


# SSC Posts Smart People Like

In [17]:
data['IQ'] = pd.to_numeric(data['IQ'], errors='coerce')

posts = []

for i in post_dummies.columns:
    corr = pearsonr(data['IQ'].fillna(data['IQ'].median()), post_dummies[i])
    posts.append([i, corr[0], corr[1]])

posts = pd.DataFrame(posts, columns=['post', 'corr', 'pvalue'])

posts[posts.pvalue < .05].sort_values('corr', ascending=False).head(20)[['post', 'corr']]

,post,corr
106,meditations on moloch,0.02769


# SSC Posts Conscientious People Like

In [18]:
data['BigFiveC'] = pd.to_numeric(data['BigFiveC'], errors='coerce')

posts = []

for i in post_dummies.columns:
    corr = pearsonr(data['BigFiveC'].fillna(data['BigFiveC'].median()), post_dummies[i])
    posts.append([i, corr[0], corr[1]])

posts = pd.DataFrame(posts, columns=['post', 'corr', 'pvalue'])

posts[posts.pvalue < .05].sort_values('corr', ascending=False).head(20)[['post', 'corr']]

,post,corr
189,why are transgender people immune to optical i...,0.027174
96,is it possible to have coherent principles aro...,0.026798
11,against tulip subsidies,0.026156
31,book review inadequate equilibria,0.025456
65,guns and states,-0.023198
20,anglophysics,-0.023574
48,control group is out of control,-0.023963
58,five case studies on politicization,-0.024270
171,thinking like a state,-0.027070
177,"universal love, said the cactus person",-0.029637


# SSC Posts Open People Like

In [19]:
data['BigFiveO'] = pd.to_numeric(data['BigFiveO'], errors='coerce')

posts = []

for i in post_dummies.columns:
    corr = pearsonr(data['BigFiveO'].fillna(data['BigFiveO'].median()), post_dummies[i])
    posts.append([i, corr[0], corr[1]])

posts = pd.DataFrame(posts, columns=['post', 'corr', 'pvalue'])

posts[posts.pvalue < .05].sort_values('corr', ascending=False).head(20)[['post', 'corr']]

,post,corr
106,meditations on moloch,0.050465
155,the goddess of everything else,0.023356
96,is it possible to have coherent principles aro...,-0.023169
118,no favourites,-0.023614
99,it’s bayes all the way up,-0.024382
16,all debates are bravery debates,-0.024448
135,review of seeing like a state,-0.025396
125,pope and change an atheist’s guide to vatican ...,-0.025396
44,contra askell on moral offsets,-0.026733
142,ssc gives a graduation speech,-0.028950


# SSC Posts Agreeable People Like

In [20]:
data['BigFiveA'] = pd.to_numeric(data['BigFiveA'], errors='coerce')

posts = []

for i in post_dummies.columns:
    corr = pearsonr(data['BigFiveA'].fillna(data['BigFiveA'].median()), post_dummies[i])
    posts.append([i, corr[0], corr[1]])

posts = pd.DataFrame(posts, columns=['post', 'corr', 'pvalue'])

posts[posts.pvalue < .05].sort_values('corr', ascending=False).head(20)[['post', 'corr']]

,post,corr
145,surfing uncertainty,0.044929
184,what is depression anyway,0.038092
51,different worlds,0.037833
3,a response to apophemi on triggers,0.031607
96,is it possible to have coherent principles aro...,0.031607
24,archipelago and atomic communitarianism,0.028337
123,pills,0.027554
165,the study of anglophysics,0.023705
161,the parable of talents,0.023323
141,society is fixed biology is mutable,0.023095


In [21]:
data['BigFiveN'] = pd.to_numeric(data['BigFiveN'], errors='coerce')

# Reverse the scale, changing emotional stability back to neuroticism
data['BigFiveN'] = data['BigFiveN'].max() - data['BigFiveN']

posts = []

for i in post_dummies.columns:
    corr = pearsonr(data['BigFiveN'].fillna(data['BigFiveN'].median()), post_dummies[i])
    posts.append([i, corr[0], corr[1]])

posts = pd.DataFrame(posts, columns=['post', 'corr', 'pvalue'])

posts[posts.pvalue < .05].sort_values('corr', ascending=False).head(20)[['post', 'corr']]

,post,corr
170,things that sometimes help if you have depression,0.055518
182,were there dark ages,0.038104
171,thinking like a state,0.035524
177,"universal love, said the cactus person",0.029313
106,meditations on moloch,0.027182
151,the categories were made for man not man for t...,0.026792
160,the nonlibertarian faq,0.025351
129,radicalizing the romanceless,0.025147
58,five case studies on politicization,0.024972
47,contra robinson on public food,0.024677
